In [1]:
import numpy as np

In [2]:
import os

In [3]:
import matplotlib.pyplot as plt

In [4]:
def stacked_images_array(path):
    
    list_of_folders = os.listdir(path)
    
    list_of_images = []

    parent_path = path

    for each_folder in list_of_folders:
    
        folder_path = os.path.join(parent_path,each_folder)
    
        list_of_images.extend(map(lambda x: plt.imread(os.path.join(folder_path,x)).reshape(1,1024),os.listdir(folder_path)))
        
    return list_of_images,list_of_folders

In [5]:
list_of_images,list_of_folders = stacked_images_array("../data/DevanagariHandwrittenDigitsDataset/Train")

In [6]:
training_data = np.concatenate(list_of_images)

In [7]:
training_data.shape

(12750, 1024)

In [10]:
list_of_folders

['digit_0',
 'digit_1',
 'digit_2',
 'digit_3',
 'digit_4',
 'digit_5',
 'digit_6',
 'digit_7',
 'digit_8',
 'digit_9']

In [8]:
unique_labels = np.arange(0,len(list_of_folders))

In [9]:
unique_labels

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [15]:
labels = []
training_data.shape[0]
len(unique_labels)
no_of_labels_for_each_character = int(training_data.shape[0]/len(list_of_folders))
labels.extend(map(lambda x: x*np.ones((no_of_labels_for_each_character,1)),unique_labels))

12750

10

In [16]:
labels = np.concatenate(labels)

In [17]:
labels.shape

(12750, 1)

In [18]:
import pandas as pd

In [19]:
training_data = pd.DataFrame(data=training_data)

In [20]:
training_data['labels'] = labels

In [21]:
training_data.head()

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,labels
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
classes_cov_mats = []

classes_cov_mats.extend(map(lambda x: np.array(training_data[training_data['labels'] == x].iloc[:,0:1024].cov()),unique_labels))

In [24]:
len(classes_cov_mats)

10

In [25]:
np.linalg.det(classes_cov_mats[0])

0.0

In [26]:
def regularized_matrices(classes_cov_mats):

    alpha = np.random.uniform(0,1)

    gamma = np.random.uniform(0,1)

    rda_cov_mats1 = []
    
    pooled_cov_mat = (no_of_labels_for_each_character-1)*sum(classes_cov_mats)/(training_data.shape[0] - len(list_of_folders))

    rda_cov_mats1.extend(map(lambda x: ((alpha*pooled_cov_mat) + ((1-alpha)*x)),classes_cov_mats))

    rda_mats1_diag_avgs = []

    rda_mats1_diag_avgs.extend(map(lambda x: np.mean(np.diag(x)),rda_cov_mats1))

    rda_cov_mats2 = []

    rda_cov_mats2.extend(map(lambda x: ((gamma*rda_cov_mats1[x]) + ((1-gamma)*rda_mats1_diag_avgs[x]*np.eye(rda_cov_mats1[0].shape[0],rda_cov_mats1[0].shape[0]))),np.arange(0,len(list_of_folders))))
    
    return rda_cov_mats2

In [27]:
rda_matrices = regularized_matrices(classes_cov_mats)

In [28]:
np.linalg.det(rda_matrices[0])

0.0

In [29]:
def PCA(cov,preserved_variance):
    
    svd_factorized_mat = np.linalg.svd(cov)
    
    eig_vals = svd_factorized_mat[1]
    
    eig_vecs = svd_factorized_mat[0]
    
    eig_vals_total = np.sum(eig_vals)
    
    eig_vals_sum = 0
    
    indices = []
    
    for i in range(0,eig_vals.shape[0]):
        
        if eig_vals_sum/eig_vals_total > preserved_variance:
            
            break
            
        eig_vals_sum += eig_vals[i]
        
        indices.append(i)
        
    return eig_vecs[:,indices]

In [30]:
preserved_variance = 0.97

In [31]:
training_data_cov = np.array(training_data.iloc[:,0:1024].cov())

In [32]:
eig_vecs_mat = PCA(training_data_cov,preserved_variance)

In [33]:
training_data_mat = np.array(training_data.iloc[:,0:1024])

In [34]:
eig_vecs_mat.shape

(1024, 199)

In [35]:
pca_reduced_data = np.matmul(training_data_mat,eig_vecs_mat)

In [36]:
pca_reduced_data.shape

(12750, 199)

In [37]:
classes_cov_mats_reduced = []

classes_cov_mats_reduced.extend(map(lambda x: np.cov(pca_reduced_data[(no_of_labels_for_each_character*x):(no_of_labels_for_each_character*(x+1)),:],rowvar=False),unique_labels))



In [38]:
len(classes_cov_mats_reduced)

10

In [39]:
correct_counts = []

accuracies = []

In [40]:
pca_reduced_rda_matrices = regularized_matrices(classes_cov_mats_reduced)

In [41]:
import scipy.stats as s

In [47]:
list_of_images,list_of_folders = stacked_images_array("../data/DevanagariHandwrittenDigitsDataset/Test")

In [48]:
testing_data = np.array(list_of_images)

In [49]:
testing_data = testing_data.reshape(testing_data.shape[0],testing_data.shape[2])

In [50]:
pca_reduced_testing_data = np.matmul(testing_data,eig_vecs_mat)

In [51]:
pca_reduced_testing_data.shape

(3000, 199)

In [52]:
class_mean_vecs = []

class_mean_vecs.extend(map(lambda x: np.mean(pca_reduced_data[no_of_labels_for_each_character*x:no_of_labels_for_each_character*(x+1),:],axis=0),np.arange(0,len(list_of_folders))))



In [53]:
len(class_mean_vecs)

10

In [54]:
len(pca_reduced_rda_matrices)

10

In [55]:
def post_prob(class_means,class_cov):
    
    p_class_on_x = s.multivariate_normal.pdf(pca_reduced_testing_data,class_means,class_cov)
    
    p_class_on_x = p_class_on_x.reshape(p_class_on_x.shape[0],1)
    
    return p_class_on_x

In [56]:
post_probs = []

post_probs.extend(map(lambda x: post_prob(class_mean_vecs[x],pca_reduced_rda_matrices[x]),np.arange(0,len(list_of_folders))))



In [57]:
posterior_probabilities = np.concatenate(post_probs,axis=1)

In [58]:
posterior_probabilities.shape

(3000, 10)

In [59]:
predicted_classes = np.argmax(posterior_probabilities,axis=1)

In [60]:
predicted_classes.shape

(3000,)

In [61]:
predicted_classes = predicted_classes.reshape(predicted_classes.shape[0],1)

In [62]:
actual_classes = []

actual_classes.extend(map(lambda x: x*np.ones((300,1)),np.arange(0,len(list_of_folders))))

In [63]:
actual_classes = np.concatenate(actual_classes)

In [64]:
actual_classes.shape

(3000, 1)

In [65]:
correct_count = np.count_nonzero(np.equal(predicted_classes,actual_classes))

In [66]:
correct_counts.append(correct_count)

In [67]:
correct_counts

[2865]

In [68]:
accuracies.append((correct_count/actual_classes.shape[0])*100)

In [69]:
accuracies

[95.5]